**Review**

Hi, my name is Dmitry and I will be reviewing your code.
  
You can find my comments in colored markdown cells:
  
<div class="alert alert-success">
  If everything is done successfully.
</div>
  
<div class="alert alert-warning">
  If I have some (optional) suggestions, or questions to think about, or general comments.
</div>
  
<div class="alert alert-danger">
  If a section requires some corrections. Work can't be accepted with red comments.
</div>
  
Please don't remove my comments, as it will make further review iterations much harder for me.
  
Feel free to reply to my comments or ask questions using the following template:
  
<div class="alert alert-info">
  For your comments and questions.
</div>
  
First of all, thank you for turning in the project! You did a great job overall, but there are some small issues that need to be fixed before the code is accepted. It should be pretty straightforward though!

Note: This project can also be found on my github [here](https://github.com/danp0kes/triple-ten-projects/blob/main/projects/final-project/final_project.ipynb)

# Introduction

In an increasingly competitive telecommunications landscape, retaining customers is paramount for sustained growth and profitability. To address this challenge, Interconnect, a prominent telecom operator, seeks to implement a proactive churn management strategy. By leveraging advanced data analytics techniques, Interconnect aims to forecast customer churn and intervene with targeted retention efforts.

This project focuses on utilizing Interconnect's rich repository of customer data, encompassing information about their plans, contracts, and demographic details. Through sophisticated predictive modeling, we aim to identify subtle patterns and indicators that precede customer churn. By detecting early signals of customer dissatisfaction or intent to switch providers, Interconnect can deploy timely interventions such as promotional codes and tailored plan options to mitigate churn risk and foster long-term customer loyalty.

## The Process

6 Main steps includes:

1. Loading Data
2. Prepare Data
3. Exploratory Data Analysis
4. Pre-Process Data
5. Create Models
6. Draw Conclusions

# 1 Load Data

Before a model can be made, the data will be briefly prepared for an analysis to understand the best course of action to take when creating the model. Loading the data entails:

1. Importing packages
2. Saving dataframes
3. Inspecting the data

## 1.1 Import Packages

Packages that enable the data to be saved, analyzed and visualized will be read in.

In [6]:
# Import Standard Libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Import preprocessing modules
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split

# Import imbalanced-learn modules
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import NearMiss

# Import models and evaluation functions
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import gridsearch and pipeline modules
from sklearn.model_selection import GridSearchCV

## 1.2 Save Dataframes

The four dataframes will be saved as `contract`, `internet`, `personal`, and `phone` using pandas.

In [7]:
#contract = pd.read_csv('/datasets/final_provider/contract.csv')
#internet = pd.read_csv('/datasets/final_provider/internet.csv')
#personal = pd.read_csv('/datasets/final_provider/personal.csv')
#phone = pd.read_csv('/datasets/final_provider/phone.csv')

contract = pd.read_csv('data/contract.csv')
internet = pd.read_csv('data/internet.csv')
personal = pd.read_csv('data/personal.csv')
phone = pd.read_csv('data/phone.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/contract.csv'

## 1.3 Inspect Data

A sample of each dataframe will be viewed. The target distribution will also be assessed.

### 1.3.1 Dataframe Samples

In [ ]:
contract.sample(5)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
157,2749-CTKAJ,2018-04-01,No,One year,No,Mailed check,20.15,419.9
6882,8065-QBYTO,2014-03-01,No,One year,Yes,Credit card (automatic),99.65,7181.25
4287,6425-YQLLO,2014-07-01,2020-01-01 00:00:00,One year,Yes,Credit card (automatic),105.95,6975.25
6910,0530-IJVDB,2014-04-01,No,Two year,Yes,Electronic check,114.60,7882.5
3964,2632-IVXVF,2014-06-01,No,Two year,No,Credit card (automatic),111.75,7511.3


In [ ]:
internet.sample(5)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
2495,6397-JNZZG,DSL,No,Yes,Yes,No,Yes,Yes
5241,2272-QAGFO,Fiber optic,No,No,No,No,No,No
4695,4482-FTFFX,DSL,No,No,Yes,Yes,No,No
4779,3317-VLGQT,Fiber optic,No,No,No,No,No,Yes
207,5046-NUHWD,DSL,Yes,No,Yes,No,No,Yes


In [ ]:
personal.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents
2759,1194-SPVSP,Male,0,No,No
6801,6668-CNMFP,Female,0,Yes,Yes
6884,7384-GHBPI,Male,0,Yes,No
4373,7632-YUTXB,Female,0,Yes,Yes
5781,1818-ESQMW,Female,0,No,No


In [ ]:
phone.sample(5)

,customerID,MultipleLines
4918,8630-QSGXK,Yes
4149,2697-NQBPF,Yes
3056,1855-AGAWH,No
5774,9894-EZEWG,No
3598,4505-EXZHB,No


### 1.3.2 Target Distribution

Determine the percentage of customers who churn.

In [ ]:
round(len(contract[contract['EndDate'] == 'No']) *100 / len(contract), 2)

73.46

# 2 Prepare Data

Preparing the data will include:

1. Merge Dataframes
2. Changing column names
3. Changing datatypes
4. Extract the target
5. Deleting redundant columns


## 2.1 Merge DataFrames

With the the `customerID` columns, merge each DataFrame. Handle appropriate missing values that may occur due to the merge.

In [ ]:
# Merge all data into one dataframe
df = contract.merge(internet, on='customerID', how='outer')\
    .merge(personal, on='customerID', how='outer')\
    .merge(phone, on='customerID', how='outer')
    
df.sample(5)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
4093,2476-YGEFM,2017-09-01,No,Month-to-month,Yes,Credit card (automatic),76.00,2215.25,Fiber optic,No,No,No,No,No,No,Female,0,No,No,Yes
5290,0774-RMNUW,2014-03-01,No,Two year,No,Bank transfer (automatic),59.70,4122.65,DSL,Yes,Yes,No,Yes,Yes,Yes,Female,0,Yes,Yes,NaN
3490,7743-EXURX,2019-03-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,51.05,415.05,DSL,No,No,Yes,No,No,No,Male,0,Yes,Yes,No
2629,5028-GZLDO,2019-09-01,2019-11-01 00:00:00,Month-to-month,No,Credit card (automatic),70.70,140.7,Fiber optic,No,No,No,No,No,No,Male,0,No,No,No
4648,9717-QEBGU,2019-12-01,No,Month-to-month,Yes,Electronic check,70.30,144,Fiber optic,No,No,No,No,No,No,Male,0,No,No,No


<div class="alert alert-success">
<b>Reviewer's comment</b>

Dataframes were merged correctly

</div>

## 2.2 Column Names

The dataframes will be editted for readability, ensuring that all column names are in snake_case. This will make it easier to reference later columns later. A function will change column names to be more readable. 

In [ ]:
# Create a function to convert column names to snake case
def to_snake_case(column_name):
    snake_case_name = ""
    prev_char = ''
    for char in column_name:
        if char.isupper() and prev_char.isupper() and prev_char:
            snake_case_name += char.lower()
        elif char.isupper() and prev_char:
            snake_case_name += "_" + char.lower()
        else:
            snake_case_name += char.lower()
        prev_char = char
    return snake_case_name

# Apply function to each dataframe
df.columns = [to_snake_case(col) for col in df.columns]

## 2.3 Data Types

Columns will be adjusted so that they fit the appropriate data types. The info will be displayed, followed by values from columns that may need to be changed.


In [ ]:
# Show contract information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
 8   internet_service   5517 non-null   object 
 9   online_security    5517 non-null   object 
 10  online_backup      5517 non-null   object 
 11  device_protection  5517 non-null   object 
 12  tech_support       5517 non-null   object 
 13  streaming_tv       5517 non-null   object 
 14  streaming_movies   5517 non-null   object 
 15  gender             7043 non-null   object 
 16  senior_citizen     7043 

In [ ]:
# Show unique values for each column
df.nunique()

customer_id          7043
begin_date             77
end_date                5
type                    3
paperless_billing       2
payment_method          4
monthly_charges      1585
total_charges        6531
internet_service        2
online_security         2
online_backup           2
device_protection       2
tech_support            2
streaming_tv            2
streaming_movies        2
gender                  2
senior_citizen          2
partner                 2
dependents              2
multiple_lines          2
dtype: int64

Three columns of interest include the following:
- `begin_date`: The date the contract began
- `end_date`: The date the contract ended
- `total_charges`: The total charges for the contract

Unique values for each will be shown

### 2.3.1 Begin Date

In [ ]:
df['begin_date'].value_counts()

begin_date
2014-02-01    366
2019-10-01    237
2019-11-01    237
2019-09-01    237
2020-01-01    233
             ... 
2020-02-01     11
2014-01-01      7
2013-10-01      3
2013-12-01      3
2013-11-01      2
Name: count, Length: 77, dtype: int64

As 77 unique dates are given, a function will be made to ensure that all dates are in the correct format. If so, this column will be converted to datetime.

In [ ]:
# Create function to check if a column ensure that all dates are in the correct format
def check_date_format(date):
    wrong_format = 0

    for i in range(len(date)):
        if len(date[i]) != 10:
            wrong_format += 1
        elif date[i][4] != '-' or date[i][7] != '-':
            wrong_format += 1
        elif not date[i][:4].isdigit():
            wrong_format += 1
        elif not date[i][5:7].isdigit():
            wrong_format += 1
            
    if wrong_format > 0:
        print('There are', wrong_format, 'dates in the wrong format')
    else:
        print('All dates are in the correct format.')

# Apply function to begin date column
check_date_format(df['begin_date'])

All dates are in the correct format.


The begin column is ready to be converted to datetime. This will be done after the end date column is also ready.

### 2.3.2 End Date Column

In [ ]:
# Repeat similar process for end_date
df['end_date'].value_counts()

end_date
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: count, dtype: int64

Note that the end date contains values that are either listed as `No` or provide a specific date. `No` values indicate churn levels, so a new column called `churn` will be made so that the `end_date` column can be converted to date-time.

In [ ]:
# Create target `churn`` column
df['churn'] = df['end_date'].apply(lambda x: 0 if x == 'No' else 1)

In [ ]:
# Convert `end_date` with 'No' to current date within the dataset
df['end_date'] = df['end_date'].apply(lambda x: df['begin_date'].max() + ' 00:00:00' if x == 'No' else x)

In [ ]:
# Convert begin and end_date to datetime
df['begin_date'] = pd.to_datetime(df['begin_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

### 2.3.3 Total Charges Column

In [ ]:
# Show total_charges values
df['total_charges'].value_counts()

total_charges
          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: count, Length: 6531, dtype: int64

The total_charges columns should be considered a float. However, this conversion likely did not happen due to missing values that could not be converted. By coercing these values, the datatype can be changed to float.

In [ ]:
# Change datatype of total charges to float
df['total_charges'] = pd.to_numeric(df['total_charges'], errors='coerce')

In [ ]:
# Show new datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           7043 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7032 non-null   float64       
 8   internet_service   5517 non-null   object        
 9   online_security    5517 non-null   object        
 10  online_backup      5517 non-null   object        
 11  device_protection  5517 non-null   object        
 12  tech_support       5517 non-null   object        
 13  streaming_tv       5517 non-null   object        
 14  streamin

## 2.4 Missing Values

From above, some values now contain missing values.

In [ ]:
# Rows with missing values
round(df.isnull().sum() / len(df) * 100, 2)

customer_id           0.00
begin_date            0.00
end_date              0.00
type                  0.00
paperless_billing     0.00
payment_method        0.00
monthly_charges       0.00
total_charges         0.16
internet_service     21.67
online_security      21.67
online_backup        21.67
device_protection    21.67
tech_support         21.67
streaming_tv         21.67
streaming_movies     21.67
gender                0.00
senior_citizen        0.00
partner               0.00
dependents            0.00
multiple_lines        9.68
churn                 0.00
dtype: float64

Columns that contain a significant percentage include:
- internet_service
- online_security
- online_backup
- device_protection
- tech_support
- streaming_tv
- streaming_movies
- multiple_lines

Interestingly, all of these apart from the multiple lines column contain the same percentage. These will be compared first and missing data in one will be compared to others. `Multiple_lines` will be dealt with at a later point.

### 2.4.1 Similar Missing Values

In [ ]:
# Find missing values in internet_service column
missing_internet_service = df[df['internet_service'].isnull()]

In [ ]:
# Determine how much of the other columns are also missing in the missing_internet_service dataframe
round(missing_internet_service.isnull().sum() / len(missing_internet_service) * 100, 2)

customer_id            0.00
begin_date             0.00
end_date               0.00
type                   0.00
paperless_billing      0.00
payment_method         0.00
monthly_charges        0.00
total_charges          0.39
internet_service     100.00
online_security      100.00
online_backup        100.00
device_protection    100.00
tech_support         100.00
streaming_tv         100.00
streaming_movies     100.00
gender                 0.00
senior_citizen         0.00
partner                0.00
dependents             0.00
multiple_lines         0.00
churn                  0.00
dtype: float64

As all other columns match up exactly with the missing internet_service column, they can be handled within the same dataframe as `maj_missing`.

In [ ]:
# Save maj_missing as missing_internet_service
maj_missing = missing_internet_service

In [ ]:
# Create dataframe that doesn't contain missing values
present = df[~df['internet_service'].isna()]

In [ ]:
# Describe the data to better understand the missing values
maj_missing.describe()

,begin_date,end_date,monthly_charges,total_charges,senior_citizen,churn
count,1526,1526,1526.000000,1520.000000,1526.000000,1526.000000
mean,2017-07-09 21:59:12.817824256,2020-01-26 13:09:49.777195264,21.079194,665.220329,0.034076,0.074050
min,2014-02-01 00:00:00,2019-10-01 00:00:00,18.250000,18.800000,0.000000,0.000000
25%,2015-09-08 12:00:00,2020-02-01 00:00:00,19.700000,159.875000,0.000000,0.000000
50%,2018-01-01 00:00:00,2020-02-01 00:00:00,20.150000,523.675000,0.000000,0.000000
75%,2019-06-01 00:00:00,2020-02-01 00:00:00,20.900000,1110.050000,0.000000,0.000000
max,2020-02-01 00:00:00,2020-02-01 00:00:00,26.900000,2006.950000,1.000000,1.000000
std,NaN,NaN,2.164221,555.158112,0.181484,0.261938


In [ ]:
present.describe()

,begin_date,end_date,monthly_charges,total_charges,senior_citizen,churn
count,5517,5517,5517.000000,5512.000000,5517.000000,5517.000000
mean,2017-04-11 01:51:58.433931520,2020-01-07 09:18:02.544861440,76.844254,2729.505406,0.197571,0.318289
min,2013-10-01 00:00:00,2019-10-01 00:00:00,23.450000,23.450000,0.000000,0.000000
25%,2015-06-01 00:00:00,2020-01-01 00:00:00,59.950000,586.837500,0.000000,0.000000
50%,2017-08-01 00:00:00,2020-02-01 00:00:00,79.450000,2139.150000,0.000000,0.000000
75%,2019-04-01 00:00:00,2020-02-01 00:00:00,94.450000,4528.000000,0.000000,1.000000
max,2020-02-01 00:00:00,2020-02-01 00:00:00,118.750000,8684.800000,1.000000,1.000000
std,NaN,NaN,21.924661,2355.681063,0.398203,0.465855


In [ ]:
# Create histogram for missing data
def histogram_comparison(missing, present):
    hist_missing = go.Histogram(x=missing['monthly_charges'], 
                                name='Missing Values', 
                                opacity=0.7, 
                                #histnorm='percent', 
                                marker_color='hotpink',
                                nbinsx=missing['monthly_charges'].nunique())

    # Create histogram for all data
    hist_present = go.Histogram(x=present['monthly_charges'], 
                            name='Present Values', 
                            opacity=0.7, 
                            #histnorm='percent', 
                            marker_color='aqua',
                            nbinsx=present['monthly_charges'].nunique())
                    

    # Create figure
    fig = go.Figure(data=[hist_missing, hist_present], layout=go.Layout(barmode='overlay'))
    

    # Update title with column name capitalized and hyphen eliminated and capitalize next word
    fig.update_layout(title=f'Comparison of Monthly Charges', barmode='overlay')


    # Update legend title
    fig.update_layout(legend_title_text='Data Availability')

    # Show figure
    fig.show()

In [ ]:
# Show histogram
histogram_comparison(maj_missing, present)

The most interesting graph that comes from this analysis is the one that shows monthly charges. Here there is a significant difference between the monthly rate where internet data is either missing or present. This suggests that internet services have not been purchased in these cases. 

In [ ]:
# Show present with monthly_charges under $25
under_25 = present[present['monthly_charges'] < 27]

# Show values count of under_25 for set columns
columns_of_interest = ['internet_service',
                       'online_security',
                       'online_backup',
                       'device_protection',
                       'tech_support',
                       'streaming_tv',
                       'streaming_movies']

# Show values count of under_25 for set columns
for column in columns_of_interest:
    print(under_25[column].value_counts())

internet_service
DSL    80
Name: count, dtype: int64
online_security
No    80
Name: count, dtype: int64
online_backup
No    80
Name: count, dtype: int64
device_protection
No    80
Name: count, dtype: int64
tech_support
No    80
Name: count, dtype: int64
streaming_tv
No    80
Name: count, dtype: int64
streaming_movies
No    80
Name: count, dtype: int64


From the above analysis, the only values that match to missing values and the monthly charges `under $25` are those with DSL as their internet service. Likewise, all of them do not have additional services as indicated by 80 counts of `No`.

Thus, these missing values will be filled with to reflect these similarities.

In [ ]:
# Fill internet_service with 'DSL' for missing values
df['internet_service'] = df['internet_service'].fillna('DSL')

# Fill all of columns of interest with 'No' for missing values
for column in columns_of_interest[1:]:
    df[column] = df[column].fillna('No')

### 2.4.2 Multiple Lines Missing Values

Follow a similar approach as above.

In [ ]:
# Investigate the missing values in multiple_lines
missing_multiple_lines = df[df['multiple_lines'].isnull()]

# Create dataframe that doesn't contain missing values
present = df[~df['multiple_lines'].isna()]

In [ ]:
histogram_comparison(missing_multiple_lines, present)

In [ ]:
# Show histogram of monthly charges for present with color on multiple_lines
fig = px.histogram(present, 
                   x='monthly_charges', 
                   color='multiple_lines', 
                   histnorm='percent', 
                   nbins=170,
                   color_discrete_map={'No':'hotpink', 'Yes':'aqua'},
                   opacity=0.7
                   )

# Create figure
fig.update_layout(title=f'Monthly Charges where Multiple Lines is Not Missing', barmode='stack')

# Update legend title
fig.update_layout(legend_title_text='Multiple Lines')

# Show figure
fig.show()

In [ ]:
# Find monthly charges under $67
under_68 = present[(present['monthly_charges'] > 26) & (present['monthly_charges'] < 68)]

# Show values count of under_68 for multiple_lines
under_68['multiple_lines'].value_counts()

multiple_lines
No     797
Yes    280
Name: count, dtype: int64

As it is not clear whether multiple lines are used or not, we cannot follow the same process for the above with similar missing values.

#### Implicit Costs

Whilst most of the customers with missing multiple lines valuees have monthly charges `under $68`, many of these customers still do have multiple lines. Instead, if we can estimate costs for each other package, an implicit cost may be created that can be used to determine whether multiple lines have been purchased or not. Costs for the following services need to be found:

- multiple_lines
- internet_service
- online_security
- online_backup
- device_protection
- tech_support
- streaming_tv
- streaming_movies

##### Multiple Lines

Using a dataframe of dsl only, split the data for multiple lines.

In [ ]:
# Estimate costs of each service to determine multiple lines
dsl_only = df[(df['internet_service'].str.contains('DSL') & 
              df['online_security'].str.contains('No') & 
              df['online_backup'].str.contains('No') & 
              df['device_protection'].str.contains('No') &
              df['tech_support'].str.contains('No') & 
              df['streaming_tv'].str.contains('No') & 
              df['streaming_movies'].str.contains('No'))]

# Create new dataframes for missing values in multiple_lines equal to 'Yes' and 'No'
multiple_lines_dsl_only = dsl_only[dsl_only['multiple_lines'] == 'Yes']
no_multiple_lines_dsl_only = dsl_only[dsl_only['multiple_lines'] == 'No']

# Compare the two dataframes with histogram of monthly charges
fig = go.Figure()

# Add traces
fig.add_trace(go.Histogram(x=multiple_lines_dsl_only['monthly_charges'], 
                           name='Multiple Lines', 
                           opacity=0.7, 
                           marker_color='aqua',
                           nbinsx=multiple_lines_dsl_only['monthly_charges'].nunique()))

fig.add_trace(go.Histogram(x=no_multiple_lines_dsl_only['monthly_charges'],
                            name='No Multiple Lines', 
                            opacity=0.7, 
                            marker_color='hotpink',
                            nbinsx=no_multiple_lines_dsl_only['monthly_charges'].nunique()))

# Update title
fig.update_layout(title=f'Monthly Charges for DSL Only Customers Without Benefits (Except Multiple Lines)', barmode='overlay')

# Update legend title
fig.update_layout(legend_title_text='Multiple Lines')

# Show figure
fig.show()

It is clear that the median difference is `$5` between the two groups. However, it is not clear why the two distinct groups exist. Instead, asking this question directly to the company may be a better way to source costs per service. This would help to determine how many lines were used. As this could not be sourced, our analysis will continue for now.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Good job dealing with missing values!

</div>

## 2.5 Delete Redundant Columns

The `total_charges` and `monthly_charges` columns may be sharing the same data and thus one of these columns may be redundant. To determine if this is the case, a new column which determines the amount of time a customer has been with the company will be made. With this new column, an estimation on the total cost will be made and compared to the actual total cost.

In [ ]:
# Create new column for number of months that the customer has been with the company
df['months_active'] = (df['end_date'] - df['begin_date']).dt.days // 30

In [ ]:
# Prove that `months_active` multiplied by `monthly_charges` equals `total_charges`
df['total_charges_calc'] = df['months_active'] * df['monthly_charges']

In [ ]:
# Show histogram of `total_charges` and `total_charges_calc`
px.histogram(df[['total_charges','total_charges_calc']], 
             title='Total Charges', 
             barmode='overlay',
             color_discrete_map={'total_charges':'hotpink', 'total_charges_calc':'aqua'},
             opacity=0.7)

Both columns are almost identical and so keeping both is redundant. As the `monthly_charges` column provides key insights, the `total_cost` column will be dropped.

In [ ]:
# Drop `total_charges_calc`, `total_charges` and months_active columns
df.drop(columns=['total_charges_calc','total_charges','months_active'], inplace=True)
#df.drop(columns=['total_charges_calc','months_active'], inplace=True)
#df.dropna(inplace=True)

In [ ]:
# Fill missing values in `multiple_lines` with 'No'
df['multiple_lines'] = df['multiple_lines'].fillna('Unknown')

# 3. Exploratory Data Analysis

Perform an analysis to understand the best way to move forward with the modelling process.

## 3.1 Target Distribution

Determine the distribution of the target. Use methods to decrease the effects of an imbalance if it occurs.

In [ ]:
# Show percentage of churn column
round(df['churn'].value_counts(normalize=True) * 100, 2)

churn
0    73.46
1    26.54
Name: proportion, dtype: float64

A model that predicts that no customer will churn will be correct 73.46% of the time.

As the target distribution is imbalanced, a number of techniques will be used to balance the dataset. These include:
- Random Over Sampling
- Random Under Sampling
- SMOTE
- ADASYN

<div class="alert alert-success">
<b>Reviewer's comment</b>

Good idea!

</div>

# 4 Pre-Process Data

Feature engineering will enrich the dataframe with additional characteristics. The data will then be encoded, before finally being scaled.

## 4.1 Feature Engineering

Create a new columns based on previous information for member duration. This will include how long the customer has been an active member.

In [ ]:
# Create new columns

df['member_duration'] = (df['end_date'] - df['begin_date']).dt.days

# Save as type object for encoding
df['month_signed'] = df['month_signed'].astype('object')
df['year_signed'] = df['year_signed'].astype('object')


## Note on Models

A number of models will be trained and the best will be picked. These will include:

- Random Forests
- Logistic Regression
- CatBoost
- LBGM

The first two models will require encoding. As categories are binomial, One Hot Encoding (OHE) will be used.

## 4.2 One Hot Encoding

In [ ]:
# Drop columns that won't be used in the model
df = df.drop(columns=['customer_id', 'begin_date', 'end_date'])

In [ ]:
# Save catgeorical columns
categorical = df.select_dtypes(include='object').columns

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   type               7043 non-null   object 
 1   paperless_billing  7043 non-null   object 
 2   payment_method     7043 non-null   object 
 3   monthly_charges    7043 non-null   float64
 4   internet_service   7043 non-null   object 
 5   online_security    7043 non-null   object 
 6   online_backup      7043 non-null   object 
 7   device_protection  7043 non-null   object 
 8   tech_support       7043 non-null   object 
 9   streaming_tv       7043 non-null   object 
 10  streaming_movies   7043 non-null   object 
 11  gender             7043 non-null   object 
 12  senior_citizen     7043 non-null   int64  
 13  partner            7043 non-null   object 
 14  dependents         7043 non-null   object 
 15  multiple_lines     7043 non-null   object 
 16  churn              7043 

<div class="alert alert-danger">
<b>Reviewer's comment</b>

THere is a problem with using `begin_date` components together with `member_duration`: the `end_date` can easily be calculated using this combination of features, which is a case of data leakage. So you either need to drop [`month_signed`, `year_signed`] or `member_duration`

</div>

<div class="alert alert-info">
Ofcourse! That makes sense. Both `month_signed` and `year_signed` have now been removed.
</div>

In [ ]:
# Convert object columns to appropriate data types
for col in ['type', 'paperless_billing', 'payment_method', 'internet_service', 
            'online_security', 'online_backup', 'device_protection', 
            'tech_support', 'streaming_tv', 'streaming_movies', 'gender', 
            'partner', 'dependents', 'month_signed', 'year_signed']:
    df[col] = df[col].astype('category')

In [ ]:
# One hot encode categorical columns, drop first column to avoid multicollinearity
df_ohe = pd.get_dummies(df, columns=categorical, drop_first=True)

<div class="alert alert-success">
<b>Reviewer's comment</b>

Categorical features were encoded

</div>

## 4.3 Split Data

Split into training, validation and test sets in 3:1:1 ratio. Split features from target. First split into 60:40 and then into 50:50 to capture appropriate ratio. Do this for both one-hot encoding and gradient boosting dataframes.

In [ ]:
# Split data into train and validation with test features and targets for ohe
X_train_ohe, X_valid_test_ohe, y_train_ohe, y_valid_test_ohe = train_test_split(df_ohe.drop(columns='churn'), df_ohe['churn'], test_size=0.4, random_state=42)
X_valid_ohe, X_test_ohe, y_valid_ohe, y_test_ohe = train_test_split(X_valid_test_ohe, y_valid_test_ohe, test_size=0.5, random_state=42)

# Repeat for gb models
X_train_gb, X_valid_test_gb, y_train_gb, y_valid_test_gb = train_test_split(df.drop(columns='churn'), df['churn'], test_size=0.4, random_state=42)
X_valid_gb, X_test_gb, y_valid_gb, y_test_gb = train_test_split(X_valid_test_gb, y_valid_test_gb, test_size=0.5, random_state=42)

NameError: name 'train_test_split' is not defined

## 4.4 Scale Features

Ensure that all feature values are scaled between 0 - 1 for efficient processing.

In [ ]:
# Fit transformer
transformer_mas = MaxAbsScaler().fit(df_ohe.to_numpy())

# Transform data for ohe, rf and lgbm dataframes
df_ohe = pd.DataFrame(transformer_mas.transform(df_ohe.to_numpy()), columns=df_ohe.columns)
df_gb = df.copy()

df_gb.drop(columns='multiple_lines', inplace=True)

<div class="alert alert-danger">
<b>Reviewer's comment</b>

If you fit the scaler using the whole dataset (including validation/test data) it can lead to data leakage. To avoid that problem scaling should be applied after splitting the data: the scaler is fit using the train set, and then both train and test are transformed. You can think of scaler as part of the model.
    
An even better approach is to use [pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) or [imblearn pipelines](https://imbalanced-learn.org/stable/references/generated/imblearn.pipeline.Pipeline.html#imblearn.pipeline.Pipeline) (if you use resampling) which helps with correctly applying scaling in cross-validation (where scaling should be applied in each fold separately)

</div>

# 5 Create Model

Four models will be created:
- Logistic Regression
- Random Forest
- LightGBM
- CatBoost

The model that results in the best AUC-ROC score will be chosen and used on the test set.

In [ ]:
# Split data into train and validation with test features and targets for ohe and gb models
X_train_ohe, X_valid_ohe, y_train_ohe, y_valid_ohe = train_test_split(df_ohe.drop(columns='churn'), df_ohe['churn'], test_size=0.2, random_state=42)
X_train_gb, X_valid_gb, y_train_gb, y_valid_gb = train_test_split(df_gb.drop(columns='churn'), df['churn'], test_size=0.2, random_state=42)

<div class="alert alert-danger">
<b>Reviewer's comment</b>

As you're using the validation set for hyperparameter tuning/model selection/comparing different balancing techniques, you need a separate test set to evaluate the final model, otherwise it won't be possible to get an unbiased estimate of its generalization performance

</div>

## 5.1 Balance Data

Create new dataframes for each sampling technique. These include:
- Random Over Sampling
- Random Under Sampling
- SMOTE
- ADASYN

In [ ]:
# Create RandomOverSampler object
ros = RandomOverSampler(random_state=42)

# Fit and transform X and y
X_ros, y_ros = ros.fit_resample(X_train_ohe, y_train_ohe)
X_train_gb_ros, y_train_gb_ros = ros.fit_resample(X_train_gb, y_train_gb)

# Create RandomUnderSampler object
rus = RandomUnderSampler(random_state=42)

# Fit and transform X and y
X_rus, y_rus = rus.fit_resample(X_train_ohe, y_train_ohe)
X_train_gb_rus, y_train_gb_rus = rus.fit_resample(X_train_gb, y_train_gb)

# Create SMOTE object
smote = SMOTE(random_state=42)

# Fit and transform X and y
X_smote, y_smote = smote.fit_resample(X_train_ohe, y_train_ohe)

# Create ADASYN object
adasyn = ADASYN(random_state=42)

# Fit and transform X and y
X_adasyn, y_adasyn = adasyn.fit_resample(X_train_ohe, y_train_ohe)


## 5.2 Create Models

Initialize models for classification:

- Logistic Regression
- Random Forests
- Gradient Boosting

Fine tune each with different hyperparameters to produce optimal AUC-ROC.

### 5.3.1 Logistic Regression

#### Balanced Data

In [ ]:
# Create Logistic Regression object
lr = LogisticRegression(random_state=42)

# Save list of balanced datasets
balanced_datasets = [[X_ros, y_ros], [X_rus, y_rus], [X_smote, y_smote], [X_adasyn, y_adasyn]]
dataset_names = ['Random Over Sampler', 'Random Under Sampler', 'SMOTE', 'ADASYN']

def best_dataset(model, balanced_datasets, dataset_names):
    best_roc = 0
    best_dataset_name = None

    for dataset, name in zip(balanced_datasets, dataset_names):
        model.fit(dataset[0], dataset[1])
        y_pred = model.predict(X_valid_ohe) 
        roc =round(roc_auc_score(y_valid_ohe, y_pred),3) 
        if roc > best_roc:
            best_roc = roc
            best_dataset_name = name


        print(f'The ROC AUC score for {name} is {roc}')

    print('\n', f'The best ROC AUC score is {best_roc} found with the dataset: {best_dataset_name}', '\n')

best_dataset(lr, balanced_datasets, dataset_names)

The ROC AUC score for Random Over Sampler is 0.831
The ROC AUC score for Random Under Sampler is 0.808
The ROC AUC score for SMOTE is 0.832
The ROC AUC score for ADASYN is 0.831

 The best ROC AUC score is 0.832 found with the dataset: SMOTE 



#### Threshold Adjustments

In [ ]:
# Save solvers
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

In [ ]:
# Create a loop to find the best f1 score with different solvers
for solver in solvers:
    model = LogisticRegression(random_state=12345, solver=solver)
    model.fit(X_train_ohe, y_train_ohe)
    probabilities_valid = model.predict_proba(X_valid_ohe)
    probabilities_one_valid = probabilities_valid[:, 1]

    best_auc_roc = 0
    best_threshold = 0
    best_solver = ''

    for threshold in np.arange(0, 0.5, 0.02):
        predicted_valid = probabilities_one_valid > threshold
        auc_roc = roc_auc_score(y_valid_ohe, predicted_valid)
        if auc_roc > best_auc_roc:
            best_auc_roc = auc_roc
            best_threshold = threshold
            best_solver = solver
            
print('Best solver:', best_solver)

print(
'Threshold = {:.2f} | , AUC-ROC = {:.3f}'.format(
    best_threshold, best_auc_roc))

Best solver: saga
Threshold = 0.32 | , AUC-ROC = 0.832


### 5.3.2 Random Forest Classifier

In [ ]:
# Set hyperparameters
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': range(7, 13, 2),
    'min_samples_split': range(2, 4, 2),
    'min_samples_leaf': range(1, 4, 2),
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=5)

# Fit the grid search to each dataset
for dataset, name in zip(balanced_datasets, dataset_names):
    grid_search.fit(dataset[0], dataset[1])

    # Get the best parameters and corresponding ROC AUC score
    best_params = grid_search.best_params_
    best_roc_auc = grid_search.best_score_
    
    # Find validation roc_auc
    y_pred = grid_search.predict(X_valid_ohe)
    roc = round(roc_auc_score(y_valid_ohe, y_pred),3)

    print("Dataset:", name)
    print("Best ROC AUC score (Training):", best_roc_auc)
    print("ROC AUC score (Validation):", roc)
    print("Best parameters:", best_params, '\n')


Dataset: Random Over Sampler
Best ROC AUC score (Training): 0.9608523462061582
ROC AUC score (Validation): 0.846
Best parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150} 

Dataset: Random Under Sampler
Best ROC AUC score (Training): 0.8918125524322994
ROC AUC score (Validation): 0.827
Best parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150} 

Dataset: SMOTE
Best ROC AUC score (Training): 0.9500987332897562
ROC AUC score (Validation): 0.82
Best parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} 

Dataset: ADASYN
Best ROC AUC score (Training): 0.9482646456610929
ROC AUC score (Validation): 0.812
Best parameters: {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150} 



### 5.3.3 Gradient Boosting Models

Two gradient boosting models will be used:

- Light GBM
- CatBoost

Hyperparameters will be saved and gridsearch will be utilized to find the best combination. All elements, including the model, balanced dataset, and hyperparameters used will be identified.

In [ ]:
# Create hyperparameters for Gradient Boosting Models
hyperparameters = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [2000, 3000],
    'eval_metric': ['AUC'],
}

In [ ]:
# Save features that are categorical
cat_features = list(df_gb.select_dtypes(include='category').columns)

# Save CatBoostClassifier model
cat = CatBoostClassifier(random_state=42, verbose=0, cat_features=cat_features)

In [ ]:
# Initialize GridSearchCV for CatBoost
grid_search_cat = GridSearchCV(estimator=cat,
                           param_grid=hyperparameters,
                           scoring='roc_auc',
                           cv=3)

# create list of categorical features
cat_features = list(df_gb.select_dtypes(include='object').columns)

balanced_datasets = [[X_train_gb_ros, y_train_gb_ros], [X_train_gb_rus, y_train_gb_rus]]
datasets_names = ['Random Over Sampler','Random Under Sampler']

for dataset, name in zip(balanced_datasets, dataset_names):
    grid_search_cat.fit(dataset[0], dataset[1])

    # Get the best parameters and corresponding ROC AUC score
    best_params_cat = grid_search_cat.best_params_
    best_roc_auc_cat = grid_search_cat.best_score_

    # Find validation roc_auc
    y_pred_cat = grid_search_cat.predict(X_valid_gb)

    roc_cat = round(roc_auc_score(y_valid_ohe, y_pred_cat),3)

    print("Dataset:", name)
    print("Best ROC AUC score (Training):", best_roc_auc_cat)
    print("ROC AUC score (Validation):", roc_cat)
    print("Best parameters:", best_params_cat, '\n')

Dataset: Random Over Sampler
Best ROC AUC score (Training): 0.9996179756387216
ROC AUC score (Validation): 0.989
Best parameters: {'eval_metric': 'AUC', 'learning_rate': 0.05, 'n_estimators': 3000} 

Dataset: Random Under Sampler
Best ROC AUC score (Training): 0.9902766528676435
ROC AUC score (Validation): 0.987
Best parameters: {'eval_metric': 'AUC', 'learning_rate': 0.1, 'n_estimators': 3000} 



In [ ]:
# Repeat for LGBM
lgbm = LGBMClassifier(random_state=42, verbose =-1, cat_features=cat_features)

grid_search_lgbm = GridSearchCV(estimator=lgbm,
                           param_grid=hyperparameters,
                           scoring='roc_auc',
                           cv=3,
)

for dataset, name in zip(balanced_datasets, dataset_names):
    grid_search_lgbm.fit(dataset[0], dataset[1])

    # Get the best parameters and corresponding ROC AUC score
    best_params_lgbm = grid_search_lgbm.best_params_
    best_roc_auc_lgbm = grid_search_lgbm.best_score_

    # Find validation roc_auc
    y_pred_lgbm = grid_search_lgbm.predict(X_valid_gb)

    roc_lgbm = round(roc_auc_score(y_valid_ohe, y_pred_lgbm),3)

    print("Dataset:", name)
    print("Best ROC AUC score (Training):", best_roc_auc_lgbm)
    print("ROC AUC score (Validation):", roc_lgbm)
    print("Best parameters:", best_params_lgbm, '\n')

Dataset: Random Over Sampler
Best ROC AUC score (Training): 0.999569456267757
ROC AUC score (Validation): 0.991
Best parameters: {'eval_metric': 'AUC', 'learning_rate': 0.01, 'n_estimators': 3000} 

Dataset: Random Under Sampler
Best ROC AUC score (Training): 0.9893202527796389
ROC AUC score (Validation): 0.983
Best parameters: {'eval_metric': 'AUC', 'learning_rate': 0.01, 'n_estimators': 2000} 



<div class="alert alert-success">
<b>Reviewer's comment</b>

Great, you tried several different models, applied various balancing techniques and tuned hyperparameters using cross-validation

</div>

# Conclusion

The model that returned the highest AUC-ROC score was the Light GBM at 0.991 on the validation set. This model used the random over sampler dataset with a learning rate of 0.01, iterated over 3000 times.

In [ ]:
# Plot ROC AUC for LBGM with its best parameters
best_params_lgbm = {'eval_metric': 'AUC', 'learning_rate': 0.01, 'n_estimators': 3000}

lgbm = LGBMClassifier(random_state=42, verbose=-1, cat_features=cat_features, **best_params_lgbm)

# Fit model on best dataset (Random Over Sampler)
lgbm.fit(X_train_gb_ros, y_train_gb_ros)

# Predict probabilities
y_pred_lgbm = lgbm.predict_proba(X_valid_gb)[:, 1]

# Calculate ROC AUC
roc_lgbm = roc_auc_score(y_valid_ohe, y_pred_lgbm)

In [ ]:
# Create figure
fig = px.line(x=[0, 1], y=[0, 1], labels={'x':'False Positive Rate', 'y':'True Positive Rate'}, title='ROC Curve')

fpr_lgbm, tpr_lgbm, _ = roc_curve(y_valid_gb, y_pred_lgbm)

# Add ROC curve for model
fig.add_scatter(x=[0] + list(fpr_lgbm) + [1],
                y=[0] + list(tpr_lgbm) + [1],
                mode='lines',
                line=dict(color='hotpink', width=2),
                name='LGBM')

# Set theme to dark
fig.update_layout(template='plotly_dark')

# Show figure
fig.show()

<div class="alert alert-danger">
<b>Reviewer's comment</b>

Two things:
    
1. Please use a separate test set to evaluate the final model
2. ROC-AUC scores like 0.99 should make you double check everything for possible data leakage :)

</div>